In [1]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
NVIDIA GeForce GTX 1060 6GB


# Yolo5 학습하기
##1. 파일 연동

In [3]:
%cd C:\Users\SM\Desktop\AI

C:\Users\SM\Desktop\AI


In [5]:
%pwd

'C:\\Users\\SM\\Desktop\\AI'

# 2. Yolov5 설치

In [7]:
!git clone https://github.com/ultralytics/yolov5
%cd C:\\Users\\SM\\Desktop\\AI\\yolov5
!pip install -qr requirements.txt

C:\Users\SM\Desktop\AI\yolov5


Cloning into 'yolov5'...


In [9]:
!pip install Pillow==10.3

   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.5 MB ? eta -:--:--
   ---------------------------------------- 2.5/2.5 MB 7.6 MB/s eta 0:00:00
  Attempting uninstall: Pillow
    Found existing installation: pillow 10.4.0
    Uninstalling pillow-10.4.0:
      Successfully uninstalled pillow-10.4.0


In [11]:
# 기존에 설치를 완료한 경우에는 해당 경로로 이동만 하면 됩니다.
%cd C:\\Users\\SM\\Desktop\\AI\\yolov5

C:\Users\SM\Desktop\AI\yolov5


#

# 3. 이미지 파일 관리

In [13]:
!mkdir C:\\Users\\SM\\Desktop\\AI\\yolov5\\Train\\labels
!mkdir C:\\Users\\SM\\Desktop\\AI\\yolov5\\Train\\images
!mkdir C:\\Users\\SM\\Desktop\\AI\\yolov5\\Val\\labels
!mkdir C:\\Users\\SM\\Desktop\\AI\\yolov5\\Val\\images

In [17]:
##검증 데이터 만들기
import os
import shutil
from sklearn.model_selection import train_test_split

def create_validation_set(train_path, val_path, split_ratio=0.3):
    """
    Train 데이터의 일부를 Val로 이동
    """
    # 필요한 디렉토리 생성
    os.makedirs(os.path.join(val_path, 'images'), exist_ok=True)
    os.makedirs(os.path.join(val_path, 'labels'), exist_ok=True)

    # Train 이미지 리스트 가져오기
    train_images = os.listdir(os.path.join(train_path, 'images'))
    train_images = [f for f in train_images if f.endswith(('.jpg', '.jpeg', '.png'))]

    # Train/Val 분할
    _, val_images = train_test_split(train_images,
                                   test_size=split_ratio,
                                   random_state=42)

    # Val로 파일 복사
    for image_file in val_images:
        # 이미지 복사
        src_image = os.path.join(train_path, 'images', image_file)
        dst_image = os.path.join(val_path, 'images', image_file)
        shutil.copy2(src_image, dst_image)

        # 라벨 파일 복사
        label_file = os.path.splitext(image_file)[0] + '.txt'
        src_label = os.path.join(train_path, 'labels', label_file)
        dst_label = os.path.join(val_path, 'labels', label_file)
        if os.path.exists(src_label):
            shutil.copy2(src_label, dst_label)

    print(f"Created validation set with {len(val_images)} images")

# 실행
train_path = 'C:/Users/SM/Desktop/AI/yolov5/Train'
val_path = 'C:/Users/SM/Desktop/AI/yolov5/Val'

create_validation_set(train_path, val_path)

Created validation set with 141 images


In [19]:
def check_dataset():
    train_path = 'C:/Users/SM/Desktop/AI/yolov5/Train'
    val_path = 'C:/Users/SM/Desktop/AI/yolov5/Val'

    # Train 데이터 확인
    train_images = len(os.listdir(os.path.join(train_path, 'images')))
    train_labels = len(os.listdir(os.path.join(train_path, 'labels')))

    # Val 데이터 확인
    val_images = len(os.listdir(os.path.join(val_path, 'images')))
    val_labels = len(os.listdir(os.path.join(val_path, 'labels')))

    print("Dataset status:")
    print(f"Train - Images: {train_images}, Labels: {train_labels}")
    print(f"Val - Images: {val_images}, Labels: {val_labels}")

# 데이터셋 상태 확인
check_dataset()

Dataset status:
Train - Images: 468, Labels: 468
Val - Images: 141, Labels: 141


# 4. 학습시작

In [21]:
#필요 라이브러리 임포트하기
import torch
import os
from IPython.display import Image, clear_output  # to display images

In [23]:
!pip install tensorflow

   ---------------------------------------- 0.0/390.3 MB ? eta -:--:--
   ---------------------------------------- 2.4/390.3 MB 12.2 MB/s eta 0:00:32
    --------------------------------------- 5.0/390.3 MB 12.1 MB/s eta 0:00:32
    --------------------------------------- 7.3/390.3 MB 11.6 MB/s eta 0:00:34
   - -------------------------------------- 10.0/390.3 MB 11.9 MB/s eta 0:00:32
   - -------------------------------------- 12.6/390.3 MB 12.0 MB/s eta 0:00:32
   - -------------------------------------- 15.2/390.3 MB 12.1 MB/s eta 0:00:31
   - -------------------------------------- 17.8/390.3 MB 12.1 MB/s eta 0:00:31
   -- ------------------------------------- 20.4/390.3 MB 12.1 MB/s eta 0:00:31
   -- ------------------------------------- 23.1/390.3 MB 12.2 MB/s eta 0:00:31
   -- ------------------------------------- 25.7/390.3 MB 12.1 MB/s eta 0:00:31
   -- ------------------------------------- 28.3/390.3 MB 12.2 MB/s eta 0:00:30
   --- ------------------------------------ 30.9/390

In [25]:
import numpy as np
import tensorflow as tf
import os
from PIL import Image
from tensorflow.python.eager.context import eager_mode

def _preproc(image, output_height=512, output_width=512, resize_side=512):
    ''' imagenet-standard: aspect-preserving resize to 256px smaller-side, then central-crop to 224px'''
    with eager_mode():
        h, w = image.shape[0], image.shape[1]
        scale = tf.cond(tf.less(h, w), lambda: resize_side / h, lambda: resize_side / w)
        resized_image = tf.compat.v1.image.resize_bilinear(tf.expand_dims(image, 0), [int(h*scale), int(w*scale)])
        cropped_image = tf.compat.v1.image.resize_with_crop_or_pad(resized_image, output_height, output_width)
        return tf.squeeze(cropped_image)

def Create_npy(imagespath, imgsize, ext) :
    images_list = [img_name for img_name in os.listdir(imagespath) if
                os.path.splitext(img_name)[1].lower() == '.'+ext.lower()]
    calib_dataset = np.zeros((len(images_list), imgsize, imgsize, 3), dtype=np.float32)

    for idx, img_name in enumerate(sorted(images_list)):
        img_path = os.path.join(imagespath, img_name)
        try:
            # 파일 크기가 정상적인지 확인
            if os.path.getsize(img_path) == 0:
                print(f"Error: {img_path} is empty.")
                continue

            img = Image.open(img_path)
            img = img.convert("RGB")  # RGBA 이미지 등 다른 형식이 있을 경우 강제로 RGB로 변환
            img_np = np.array(img)

            img_preproc = _preproc(img_np, imgsize, imgsize, imgsize)
            calib_dataset[idx,:,:,:] = img_preproc.numpy().astype(np.uint8)
            print(f"Processed image {img_path}")

        except Exception as e:
            print(f"Error processing image {img_path}: {e}")

    np.save('calib_set.npy', calib_dataset)


In [27]:
# "cannot identify image file" 에러가 발생하는 경우, PILLOW Version을 "!pip install Pillow==10.1" 로 변경하여 설치합니다.
Create_npy('C:/Users/SM/Desktop/AI/yolov5/Train/images', 512, 'jpg')

In [29]:
# 기존에 설치를 완료한 경우에는 해당 경로로 이동만 하면 됩니다.
%cd C:\\Users\\SM\\Desktop\\AI\\yolov5

C:\Users\SM\Desktop\AI\yolov5


In [35]:
import subprocess
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

command = ["python", "train.py", "--img", "512", "--batch", "16", "--epochs", "100", "--data", "C:\\Users\\SM\\Desktop\\AI\\yolov5\\data.yaml", "--weights", "yolov5m.pt", "--cache", "--workers", "0"]
process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, encoding='utf-8')

for line in process.stdout:
    print(line, end="")


train: weights=yolov5m.pt, cfg=, data=C:\Users\SM\Desktop\AI\yolov5\data.yaml, hyp=data\hyps\hyp.scratch-low.yaml, epochs=100, batch_size=16, imgsz=512, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data\hyps, resume_evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=0, project=runs\train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
github: up to date with https://github.com/ultralytics/yolov5 
YOLOv5  v7.0-383-g1435a8ee Python-3.12.7 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce GTX 1060 6GB, 6144MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_

# 5. yolov5모델 학습 결과 검증

In [39]:
%tensorboard --logdir runs --port=6007

UsageError: Line magic function `%tensorboard` not found.


In [41]:
%load_ext tensorboard
%tensorboard --logdir runs

In [37]:
!mkdir C:\\Users\\SM\\Desktop\\AI\\yolov5\\Sample

In [51]:
import subprocess

# 명령어 설정
command = [
    "python", "detect.py",
    "--weights", "C:\\Users\\SM\\Desktop\\AI\\yolov5\\runs\\train\\exp3\\weights\\best.pt",
    "--img", "512",
    "--conf", "0.2",
    "--source", "C:\\Users\\SM\\Desktop\\AI\\yolov5\\Sample"
]
# subprocess를 사용하여 실시간으로 명령 실행
with subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True) as process:
    for line in process.stdout:
        print(line, end="")  # 실시간으로 출력

detect: weights=['C:\\Users\\SM\\Desktop\\AI\\yolov5\\runs\\train\\exp3\\weights\\best.pt'], source=C:\Users\SM\Desktop\AI\yolov5\Sample, data=data\coco128.yaml, imgsz=[512, 512], conf_thres=0.2, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5  v7.0-383-g1435a8ee Python-3.12.7 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce GTX 1060 6GB, 6144MiB)

Fusing layers... 
Model summary: 212 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
video 1/1 (1/1745) C:\Users\SM\Desktop\AI\yolov5\Sample\[VAP].mp4: 512x288 1 sign, 26.9ms
video 1/1 (2/1745) C:\Users\SM\Desktop\AI\yolov5\Sample\[VAP].mp4: 512x288 1 sign, 12.9ms
video 1/1 (3/1745) C:\Users\SM\Desktop\AI\yolov5\Sample\[VAP

In [ ]:
#display inference on ALL test images

import glob
from IPython.display import Image, display

for imageName in glob.glob('C:/Users/SM/Desktop/AI/yolov5/runs/detect/exp3/*.jpg')[:10]: #이미지 파일 형식에 맞춰 .png 또는 .jpg 등으로 수정
    display(Image(filename=imageName))
    print("\n")